# APIM Costing Sample - Clean Up

This notebook removes all resources created by the costing sample.

⚠️ **Warning**: This will delete Azure resources. Use with caution!

## ⚙️ Initialize Configuration

Set the same parameters used during deployment.

In [ ]:
import sys
from pathlib import Path

# APIM Samples imports
import azure_resources as az
from apimtypes import INFRASTRUCTURE
from console import print_error, print_info, print_ok, print_val, print_warning

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

# Match the configuration from create.ipynb
infrastructure = INFRASTRUCTURE.SIMPLE_APIM
index = 2
sample_index = 1  # Match the index used during deployment

# Optional: Use your own existing APIM deployment (uncomment both lines below)
# existing_rg_name = 'your-resource-group-name'
# existing_apim_name = 'your-apim-service-name'

# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

# Determine resource group name and APIM name
if 'existing_rg_name' in dir() and 'existing_apim_name' in dir():
    rg_name = existing_rg_name
    apim_name = existing_apim_name
    print_info(f'Using existing APIM: {apim_name} in {rg_name}')
else:
    rg_name = az.get_infra_rg_name(infrastructure, index)
    # Auto-detect APIM name from the resource group
    apim_result = az.run(
        f'az apim list --resource-group {rg_name} --query "[0].name" -o tsv',
        log_command=False
    )
    apim_name = apim_result.text.strip() if apim_result.success and apim_result.text.strip() else None
    if not apim_name:
        print_error(f'No APIM instance found in resource group: {rg_name}')
        sys.exit(1)
    print_info(f'Using infrastructure: {infrastructure.value} (index: {index})')

subscription_id_output = az.run('az account show --query id -o tsv', log_command=False)
subscription_id = subscription_id_output.text.strip() if subscription_id_output.success else None

config = {
    'rg_name': rg_name,
    'apim_name': apim_name,
    'subscription_id': subscription_id,
    'sample_index': sample_index,
    'deployment_name': f'costing-{sample_index}'
}

print_ok('Configuration loaded')
print_val('Resource Group', config['rg_name'])
print_val('APIM Service', config['apim_name'])
print_val('Sample Index', config['sample_index'])
print_val('Deployment Name', config['deployment_name'])

## 📋 List Resources to Delete

Query the deployment to see what will be deleted.

In [ ]:
print_info(f'Checking deployment: {config["deployment_name"]}')

deployment_result = az.run(
    f'az deployment group show --resource-group {config["rg_name"]} --name {config["deployment_name"]} -o json',
    log_command=False
)

if not deployment_result.success:
    print_warning('Deployment not found')
    print_info('Resources may have already been deleted or deployment name is incorrect')
else:
    outputs = deployment_result.json_data.get('properties', {}).get('outputs', {})

    config['app_insights_name'] = outputs.get('applicationInsightsName', {}).get('value')
    config['log_analytics_name'] = outputs.get('logAnalyticsWorkspaceName', {}).get('value')
    config['storage_account_name'] = outputs.get('storageAccountName', {}).get('value')
    config['workbook_id'] = outputs.get('workbookId', {}).get('value')
    config['cost_export_name'] = f"apim-cost-export-{config['sample_index']}-{config['rg_name']}"

    print_ok('Found the following resources to delete:')
    print()
    if config.get('app_insights_name'):
        print_val('  Application Insights', config['app_insights_name'])
    if config.get('log_analytics_name'):
        print_val('  Log Analytics', config['log_analytics_name'])
    if config.get('storage_account_name'):
        print_val('  Storage Account', config['storage_account_name'])
    if config.get('workbook_id'):
        print_val('  Workbook ID', config['workbook_id'])
    print_val('  Cost Export', config['cost_export_name'])
    print_val('  Deployment', config['deployment_name'])

## 🗑️ Delete Sample API Resources

Delete the sample API and subscriptions created for cost tracking.

In [ ]:
api_id = 'cost-tracking-api'
business_units = ['bu-hr', 'bu-finance', 'bu-marketing', 'bu-engineering']

print_info('Deleting sample API subscriptions...')
for sub_id in business_units:
    result = az.run(
        f'az apim api subscription delete '
        f'--resource-group {config["rg_name"]} '
        f'--service-name {config["apim_name"]} '
        f'--subscription-id {sub_id} '
        f'--yes',
        log_command=False
    )
    if result.success:
        print_ok(f'  Deleted subscription: {sub_id}')
    else:
        print_warning(f'  Subscription not found: {sub_id}')

print_info('Deleting sample API...')
result = az.run(
    f'az apim api delete '
    f'--resource-group {config["rg_name"]} '
    f'--service-name {config["apim_name"]} '
    f'--api-id {api_id} '
    f'--delete-revisions true '
    f'--yes',
    log_command=False
)

if result.success:
    print_ok(f'Deleted API: {api_id}')
else:
    print_warning(f'API not found: {api_id}')

## 🗑️ Delete Infrastructure Resources

Delete Application Insights, Log Analytics, and Storage Account.

In [ ]:
print_info('Deleting infrastructure resources...')
print()

# Delete Storage Account
if config.get('storage_account_name'):
    print_info(f'Deleting Storage Account: {config["storage_account_name"]}')
    result = az.run(
        f'az storage account delete '
        f'--name {config["storage_account_name"]} '
        f'--resource-group {config["rg_name"]} '
        f'--yes'
    )
    if result.success:
        print_ok('  Storage Account deleted')
    else:
        print_warning('  Storage Account not found or already deleted')

# Delete Application Insights
if config.get('app_insights_name'):
    print_info(f'Deleting Application Insights: {config["app_insights_name"]}')
    result = az.run(
        f'az monitor app-insights component delete '
        f'--app {config["app_insights_name"]} '
        f'--resource-group {config["rg_name"]}',
        log_command=False
    )
    if result.success:
        print_ok('  Application Insights deleted')
    else:
        print_warning('  Application Insights not found or already deleted')

# Delete Log Analytics Workspace
if config.get('log_analytics_name'):
    print_info(f'Deleting Log Analytics: {config["log_analytics_name"]}')
    result = az.run(
        f'az monitor log-analytics workspace delete '
        f'--workspace-name {config["log_analytics_name"]} '
        f'--resource-group {config["rg_name"]} '
        f'--yes '
        f'--force true',
        log_command=False
    )
    if result.success:
        print_ok('  Log Analytics Workspace deleted')
    else:
        print_warning('  Log Analytics Workspace not found or already deleted')

# Delete Workbook
if config.get('workbook_id'):
    print_info('Deleting Azure Monitor Workbook')
    result = az.run(
        f'az resource delete --ids {config["workbook_id"]}',
        log_command=False
    )
    if result.success:
        print_ok('  Workbook deleted')
    else:
        print_warning('  Workbook not found or already deleted')

print()
print_ok('Infrastructure resources cleanup complete')

## 🗑️ Delete Cost Management Export

Delete the Cost Management export (subscription-scoped resource).

In [ ]:
print_info('Deleting Cost Management export...')

result = az.run(
    f'az costmanagement export delete '
    f'--name {config["cost_export_name"]} '
    f'--scope "/subscriptions/{config["subscription_id"]}" '
    f'--yes',
    log_command=False
)

if result.success:
    print_ok(f'Cost export deleted: {config["cost_export_name"]}')
else:
    print_warning('Cost export not found or already deleted')

## ✅ Cleanup Complete

All costing sample resources have been deleted.

In [ ]:
print()
print_ok('='*60)
print_ok('Cleanup complete!')
print_ok('='*60)
print()
print_info('All costing sample resources have been removed')
print_info('You can now run create.ipynb again to deploy fresh resources')